In [8]:
import os


class Para:
    read_folder_path = 'data/sq_space_1d_rate_20d_12-21_raw'
    write_folder_path = 'data/sq_space_1d_rate_20d_12-21_pre'

    feature_column_start_name = 'ep_ratio_ttm'
    feature_column_end_name = 'BR'

    month_num = len(os.listdir(read_folder_path))
para = Para()
print(para.month_num)

2411


In [9]:
import time
import numpy as np
import pandas as pd


time_begin = time.time()

df = pd.DataFrame()

for i_csv in range(para.month_num):

    df_curr_month = pd.read_csv(para.read_folder_path +'/'+ str(i_csv) + '.csv', index_col='order_book_id')

    df_curr_month['date'] = pd.to_datetime(df_curr_month['date'])

    df = pd.concat([df, df_curr_month])

df

,board_type,sector_code,yield_rate,month,date,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,...,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
order_book_id,,,,,,,,,,,,,,,,,,,,,
600104.XSHG,MainBoard,ConsumerDiscretionary,0.100989,0,2012-01-04,0.111687,2.182483,2.332016,-0.086936,-0.231442,...,59.641256,55.555556,4.737601,0.369354,0.363153,0.337286,0.278390,0.725711,100.789889,98.615385
600104.XSHG,MainBoard,ConsumerDiscretionary,0.076447,1,2012-01-05,0.109902,2.217933,2.294742,-0.031787,-0.191511,...,61.702128,66.666667,5.894473,0.384654,0.368433,0.341246,0.292549,0.725674,115.268456,106.825397
600104.XSHG,MainBoard,ConsumerDiscretionary,0.088024,2,2012-01-06,0.111373,2.188648,2.325447,-0.003374,-0.153884,...,61.965812,55.555556,4.162846,0.380515,0.366357,0.340036,0.301877,0.722134,111.746032,98.825257
600104.XSHG,MainBoard,ConsumerDiscretionary,0.057719,3,2012-01-09,0.106141,2.296540,2.216198,0.074765,-0.108154,...,70.529801,55.555556,8.775004,0.387240,0.370256,0.341280,0.350016,0.715151,138.928571,123.228995
600104.XSHG,MainBoard,ConsumerDiscretionary,0.017710,4,2012-01-10,0.103705,2.350485,2.165334,0.163054,-0.053912,...,70.333333,66.666667,10.595402,0.400665,0.376082,0.342909,0.409107,0.709062,143.830571,116.720779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600104.XSHG,MainBoard,ConsumerDiscretionary,0.035839,2406,2021-11-29,0.102816,0.873245,3.309329,-0.070114,0.081087,...,25.352113,44.444444,-4.392519,0.502343,0.460402,0.439208,0.812255,0.800774,93.882353,91.043084
600104.XSHG,MainBoard,ConsumerDiscretionary,0.044612,2407,2021-11-30,0.103538,0.867160,3.332552,-0.121083,0.040653,...,30.857143,33.333333,-4.997738,0.499667,0.455209,0.439263,0.828216,0.777241,92.108363,95.316159
600104.XSHG,MainBoard,ConsumerDiscretionary,0.019812,2408,2021-12-01,0.102307,0.877592,3.292938,-0.140491,0.004424,...,34.594595,44.444444,-3.854854,0.494703,0.447822,0.439180,0.828216,0.765062,99.118388,98.284314


In [10]:
# Replace infinite updated data with nan
df.replace(to_replace=[np.inf, -np.inf], value=np.nan, inplace=True)

# 剔除含有 na 的行
df_dropna = df.dropna(axis=0, how='any')

# 选择特征列和标签列
df_dropna_y = df_dropna[['board_type', 'sector_code', 'month', 'date']]
df_dropna_features = df_dropna.loc[:, para.feature_column_start_name:para.feature_column_end_name]
df_dropna_features = pd.concat([df_dropna['yield_rate'], df_dropna_features], axis=1)

# 中位数去极值
df_dropna_features = df_dropna_features.clip(
    lower=df_dropna_features.median(axis=0) - 5 * df_dropna_features.mad(axis=0),
    upper=df_dropna_features.median(axis=0) + 5 * df_dropna_features.mad(axis=0),
    axis=1)

# 标准化
df_dropna_features = (df_dropna_features - df_dropna_features.mean()) / df_dropna_features.std(ddof=0)

# 拼接标签列和特征列
df_dropna_standard = pd.concat([df_dropna_y, df_dropna_features], axis=1)

# 写入文件
df_dropna_standard.to_csv(para.write_folder_path + '/0.csv')

print()
print('Done!')


Done!
